In [147]:
import numpy as np
import matplotlib.pyplot as plt

step_size = 0.5
hbar = 1.054571817E-34
elm = 9.1093837015E-31
# Constants
eV_to_J = 1.60218e-19  # Conversion factor from eV to Joules

# Convert grid step to meters for calculations
grid_step_m = step_size * 1e-10  # 1 Å = 1e-10 meters

# Calculating the maximum momentum to avoid aliasing
# Corresponds to a de Broglie wavelength of twice the grid step size
lambda_min = 2 * grid_step_m  # Minimum wavelength
p_max = hbar * (2 * np.pi) / lambda_min  # Maximum momentum using de Broglie wavelength

# Calculating the maximum energy (upper bound Eb) using E = p^2 / (2*m)
Eb_J = p_max**2 / (2 * elm)  # Energy in Joules
Eb_eV = Eb_J / eV_to_J  # Convert energy to eV

# Lower energy bound (Ea) - set to zero as per the problem context
Ea_eV = 0

# Display the results
print(f'Lower energy bound (Ea) = {Ea_eV} eV')
print(f'Upper energy bound (Eb) = {Eb_eV} eV')



Lower energy bound (Ea) = 0 eV
Upper energy bound (Eb) = 150.41174886354125 eV


In [148]:
def spectra(step, E, gam, pos, V, dx, ekinscale):
    n = len(pos)
    G0 = np.zeros((n, n), dtype=complex)
    k0 = np.sqrt((E + 1j * gam) / ekinscale)
    k1 = np.sqrt((E + 1j * gam - step) / ekinscale)
    tb = 2 * k0 / (k0 + k1)
    rb = (k0 - k1) / (k1 + k0)
    W = np.zeros((n, n))
    for i in range(n):
        phi0k = tb * np.exp(1j * k1 * pos[i])
        W[i, i] = dx * V[i] / ekinscale
        for j in range(n):
            G0[i,j] = Greenf(step, pos[i], pos[j], E, gam, ekinscale)
    

    # T = np.eye(n) - G0 * W
    # print(G0*W)
    print(G0)
    phi0k_transposed = np.transpose(phi0k)
    phisol = T / phi0k_transposed
    
    phirefl = rb * np.exp(-1j * k0 * (pos[0] - 1)) + outside_sol(pos[0] - 1, pos, V, dx, phisol, step, E, gam, ekinscale)
    phitrans = tb * np.exp(1j * k1 * (pos[n - 1] + 1)) + outside_sol(pos[n - 1] + 1, pos, V, dx, phisol, step, E, gam, ekinscale)

    tra = (np.real(k1) / np.real(k0)) * np.abs(phitrans)**2
    refl = (np.abs(phirefl / (np.exp(1j * k0 * (pos[0] - 1)))))**2
    absor = 1 - (refl + tra)
    return refl, tra, absor
def Greenf(step, x, xp, E, gam, ekinscale):
    Gf = 0
    k0 = np.sqrt((E + 1j * gam) / ekinscale)
    k1 = np.sqrt((E + 1j * gam - step) / ekinscale)
    if x >= 0 and xp >= 0:
        Gf = np.exp(1j * k1 * abs(x - xp)) / (2j * k1) + np.exp(1j * k1 * (x + xp)) * ((k1 - k0) / (k1 + k0)) / (2j * k1)
    elif x < 0 and xp >= 0:
        Gf = np.exp(-1j * k0 * x + 1j * k1 * xp) / (1j * (k0 + k1))
    elif xp < 0 and x >= 0:
        Gf = np.exp(-1j * k0 * xp + 1j * k1 * x) / (1j * (k0 + k1))
    return Gf

def outside_sol(x, pos, V, dx, phisol, step, E, gam, ekinscale):
    extra = 0
    n = len(pos)
    for i in range(n):
        extra += Greenf(step, x, pos[i], E, gam, ekinscale) * dx * (V[i] / ekinscale) * phisol[i]
    return extra

In [149]:
# Constants
E_min = 0  # in eV
E_max = 0.3  # in eV
deltaE = 0.0005  # in eV
step_size = 0.5
x1_min = 0
x1_max = 80
tau = 1e-9  # 1 ns in seconds
qel = 1.602176634E-19
gam = (hbar * 2 * np.pi / tau) / qel  # damping factor
recipunit = 1.0E+10
ekinscale = (hbar * recipunit)**2 / (2 * elm) / qel

n = int((x1_max - x1_min) / step_size)
U = np.zeros(n)  # Potential barrier
x1 = np.zeros(n)

for i in range(n):
    x1[i] = x1_min + step_size / 2 + (i - 1) * step_size
    if x1[i] <= 15:
        U[i] = 0.2
    elif 65 <= x1[i] <= 80:
        U[i] = 0.2
    else:
        U[i] = 0
edivide = round((E_max - E_min) / deltaE)
E0 = np.zeros(edivide)

for i in range(edivide):
    E0[i] = deltaE / 2 + E_min + (i - 1) * deltaE

R_values = []
T_values = []
A_values = []

for i in range(edivide):
    R, T, A = spectra(0, E0[i], gam, x1, U, step_size, ekinscale)  # the step is set to 0 to look at the constant background case
    R_values.append(R)
    T_values.append(T)
    A_values.append(A)
    print(i)
plt.figure(figsize=(8, 6))
plt.plot(E0, R_values, label='Reflection', linewidth=2)
plt.plot(E0, T_values, label='Transmission', linewidth=2)
plt.plot(E0, A_values, label='Absorption', linewidth=2)
plt.xlabel('Energy (eV)')
plt.ylabel('Coefficient')
plt.title('Unbiased junction')
plt.legend(fontsize=10, loc='upper left')
plt.legend(bbox_to_anchor=(0.25, 0.5, 0.1, 0.1))
plt.gca().set_position(plt.gca().get_position() + [0.05, 0, 0, 0])
plt.savefig('unbiased_junction_spectra.png')  # You can specify other file formats like '.jpg', '.pdf', etc.
plt.show()


[[  0.        +0.j         -61.46922504-0.51045713j
  -61.22073915-0.51044465j ... -32.67544129-0.44212617j
  -32.54334696-0.44142922j -32.41178659-0.44073067j]
 [-61.46922504-0.51045713j -61.71871943-0.51046131j
  -61.46922504-0.51045713j ... -32.80807176-0.44282152j
  -32.67544129-0.44212617j -32.54334696-0.44142922j]
 [-61.22073915-0.51044465j -61.46922504-0.51045713j
  -61.71871943-0.51046131j ... -32.94124055-0.44351522j
  -32.80807176-0.44282152j -32.67544129-0.44212617j]
 ...
 [-32.67544129-0.44212617j -32.80807176-0.44282152j
  -32.94124055-0.44351522j ... -61.71871943-0.51046131j
  -61.46922504-0.51045713j -61.22073915-0.51044465j]
 [-32.54334696-0.44142922j -32.67544129-0.44212617j
  -32.80807176-0.44282152j ... -61.46922504-0.51045713j
  -61.71871943-0.51046131j -61.46922504-0.51045713j]
 [-32.41178659-0.44073067j -32.54334696-0.44142922j
  -32.67544129-0.44212617j ... -61.22073915-0.51044465j
  -61.46922504-0.51045713j -61.71871943-0.51046131j]]


IndexError: invalid index to scalar variable.